# First look
The idea here should be to load in the data, have a quick look, and develop some of the initial processing steps to purely clean up the data, not necessarily get it in the state for the final modelling step to tag.

I think it would be good to move all ready code into the "library" so that we can just pull it from there. Should aim to make it as light as possible, because we should be saving, in a database, step by step what we do

Another aspect would be to find errors

We can leave everything we find in the next block

## Findings

...

In [1]:
from pathlib import Path
import pandas as pd
from basic_analysis import cloud_n_count
import re
from collections import Counter

data_loc = Path('/home/andreas/Desktop/Projects/Football/Data/Saved_stories')
save_loc = Path('/home/andreas/Dropbox/Projects/Football/Scripts/Dev/Tagger/data')

pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
stories = pd.read_csv(data_loc / 'stories.csv')

In [3]:
stories.shape

(19991, 10)

In [4]:
stories.head()

,article_date,article_image,article_link,article_summary,article_title,story_author,story_date,story_keywords,story_text,story_twitter
0,2019-03-30T10:30:00,Son Hueng-min gives the thumbs up,http://www.telegraph.co.uk/football/2019/03/30...,NaN,Son Hueng-min: Tottenham players must now live...,NaN,NaN,"['Tottenham Hotspur FC,Son Heung-min,Sport,Foo...",\nFollow your club now for first access to all...,"They have one of the finest training grounds, ..."
1,2019-03-30T05:07:14,Burnley's Dwight McNeil celebrates scoring his...,http://www.telegraph.co.uk/football/2019/03/30...,NaN,Burnley boost survival hopes on landmark day f...,NaN,NaN,"['Wolverhampton Wanderers FC,Sport,Football,Pr...",\nFollow your club now for first access to all...,Dwight McNeil capped the sort of week that sho...
2,2019-03-30T05:06:08,Wes Morgan scores with an improbable backheel,http://www.telegraph.co.uk/football/2019/03/30...,NaN,Wes Morgan marks Leicester return with backhee...,NaN,NaN,"['Sport,Leicester City FC,Football,Premier Lea...",\nFollow your club now for first access to all...,The &pound;9m compensation fee Leicester paid ...
3,2019-03-31T08:00:00,"Pochettino, Torreira and Hazard",http://www.telegraph.co.uk/football/2019/03/31...,NaN,What missing out on the Premier League top fou...,NaN,NaN,"['Tottenham Hotspur FC,Sport,Football,Standard...",\nFollow your club now for first access to all...,"With eight games left to go for Tottenham, Ars..."
4,2019-03-30T10:30:00,James Forrest in action at Ibrox,http://www.telegraph.co.uk/football/2019/03/30...,NaN,James Forrest reveals hurt of defeat by Ranger...,NaN,NaN,"['Sport,Football,Standard,Steven Gerrard,Brend...",It was to be the last Old Firm collision of Br...,It was to be the last Old Firm collision of Br...


In [5]:
print(stories.shape)
print(stories.drop_duplicates().shape)
print(stories.article_link.nunique())
print(stories.article_title.nunique())

condition = stories.article_link.str.contains('fake_link').fillna(False)
print(stories[~ condition].article_link.nunique())

(19991, 10)
(19060, 10)
17913
18465
17864


Interesting that we have some duplicates - do some value counts

In [6]:
stories[~ condition].article_link.value_counts().head()

https://www.dailymail.co.uk/sport/football/article-6559019/Father-Cardiff-City-youth-player-slams-toxic-Craig-Bellamy.html    1
https://www.theguardian.com/football/2018/nov/15/alexis-sanchez-manchester-united-10percent-wage-bill-rise                    1
http://www.bbc.com/sport/football/47064928                                                                                    1
https://www.theguardian.com/football/2016/mar/19/inverness-hearts-st-johnstone-scottish-premiership                           1
https://www.dailymail.co.uk/sport/football/article-6868569/Solskjaer-bold-choice-works-United-worry-results-money.html        1
Name: article_link, dtype: int64

In [3]:
stories['num_duplicates'] = stories.groupby('article_title')['article_link'].transform('count')

In [4]:
bad_titles = [
    'Who was your man of the match', 
    "The Dozen: the weekend",
    'Who is in Garth'
]
regex = '(?:' + '|'.join(bad_titles) + ')'

has_bad_title = stories.article_title.str.contains(regex).fillna(False)
has_empty = (stories.num_duplicates > 1) & (stories.story_text.isna())

condition = (~ has_bad_title) & (~ has_empty) & (stories.article_title.notna())

In [5]:
# Pretty OK with what is here
reduced_stories = stories[condition].reset_index(drop = True)

Now need to do some things like cleaning up basic stuff that is included and don't want - this will typically be basic text that is repeated a lot. This would typically be in the text, not sure what would find in the title or whatever.

* Maybe should take the texts and find the most common bigrams / trigrams
* Then eliminate what looks bad

In [43]:
_ = cloud_n_count(reduced_stories.article_title.tolist(), most_common = 30, n_gram = 2, most_common_ng = 30)
_ = cloud_n_count(reduced_stories.article_title.tolist(), most_common = 0, n_gram = 3, most_common_ng = 30)
_ = cloud_n_count(reduced_stories.article_title.tolist(), most_common = 0, n_gram = 6, most_common_ng = 30)

The 30 most common words are the following:
[("'", 9375), ('to', 7048), ('-', 6609), ('the', 4385), ('s', 4152), ('in', 3542), ('for', 3224), ('and', 2954), ('of', 2952), ('as', 2904), (':', 2685), ('a', 2282), (',', 2262), ('United', 2063), ('on', 2058), ('at', 2034), ('Manchester', 2031), ('with', 1821), ('’', 1812), ('League', 1792), ('City', 1762), ('after', 1739), ('is', 1518), ("':", 1410), ('Liverpool', 1293), ('1', 1270), ('Chelsea', 1231), ('he', 1224), ('his', 1092), ('Premier', 1064)]

The 30 most common ngrams are the following:
[(("'", 's'), 3002), (('Manchester', 'United'), 1176), (('’', 's'), 1137), (('Premier', 'League'), 1061), (('Manchester', 'City'), 825), (('-', '0'), 624), (('-', '1'), 600), (('in', 'the'), 580), (('1', '-'), 565), (('of', 'the'), 514), (('West', 'Ham'), 445), (('0', '-'), 424), (('2', '-'), 423), (('FA', 'Cup'), 422), (("'", 't'), 395), (('-', '2'), 394), (("'", 'I'), 372), (('Real', 'Madrid'), 368), (('Ole', 'Gunnar'), 359), (('Champions', 'Leagu

In [44]:
c = reduced_stories.article_summary.notna()
_ = cloud_n_count(reduced_stories.loc[c, 'article_summary'].tolist(), most_common = 30, n_gram = 2, most_common_ng = 30)
_ = cloud_n_count(reduced_stories.loc[c, 'article_summary'].tolist(), most_common = 0, n_gram = 3, most_common_ng = 30)
_ = cloud_n_count(reduced_stories.loc[c, 'article_summary'].tolist(), most_common = 0, n_gram = 6, most_common_ng = 30)

The 30 most common words are the following:
[('the', 13388), ('.', 11575), ('to', 8457), ('a', 7003), ('-', 6966), ('of', 5834), ('in', 5368), (',', 5299), ('s', 5094), ("'", 5077), ('and', 4946), ('at', 3374), ('on', 3260), ('for', 3133), ('his', 3051), ('League', 2735), ('with', 2650), ('as', 2423), ('has', 2275), ('is', 2229), ('their', 2067), ('from', 1882), ('after', 1871), ('he', 1717), ('United', 1695), ('’', 1653), ('The', 1621), ('Manchester', 1609), ('by', 1535), ('City', 1503)]

The 30 most common ngrams are the following:
[(("'", 's'), 3780), (('in', 'the'), 1800), (('of', 'the'), 1747), (('’', 's'), 1297), (('Premier', 'League'), 1235), (('Manchester', 'United'), 919), (('at', 'the'), 853), (('-', '0'), 760), (('for', 'the'), 716), (('to', 'the'), 701), (('Manchester', 'City'), 647), (('the', 'club'), 630), (('the', 'Premier'), 598), (('with', 'a'), 561), (('game', 'between'), 558), (('coverage', 'of'), 556), (('-', '1'), 546), (('-', 'year'), 529), (('2', '-'), 483), (('t

In [45]:
c = reduced_stories.story_text.notna()
_ = cloud_n_count(reduced_stories.loc[c, 'story_text'].tolist(), most_common = 30, n_gram = 2, most_common_ng = 30)
_ = cloud_n_count(reduced_stories.loc[c, 'story_text'].tolist(), most_common = 0, n_gram = 3, most_common_ng = 30)
_ = cloud_n_count(reduced_stories.loc[c, 'story_text'].tolist(), most_common = 0, n_gram = 6, most_common_ng = 30)

The 30 most common words are the following:
[('.', 582050), ('the', 555483), (',', 416804), ('to', 294252), ('a', 258524), ('and', 224110), ('in', 212158), ('of', 187502), ('-', 180355), ('s', 136945), ("'", 136155), ('for', 109788), ('on', 107703), ('is', 99932), ('with', 96892), ('’', 88813), ('at', 85334), ('was', 82518), ('his', 79726), ('that', 74857), ('from', 73598), ('he', 72424), ('it', 66329), ('have', 64300), ('I', 64093), ('(', 62959), ('by', 57536), ('but', 56247), ('The', 55821), ('as', 55469)]

The 30 most common ngrams are the following:
[(('in', 'the'), 79531), (("'", 's'), 77432), (('’', 's'), 59247), (('of', 'the'), 58255), (('.', 'The'), 41235), (('to', 'the'), 26117), (('on', 'the'), 24290), ((',', 'but'), 23471), (('at', 'the'), 23408), (('for', 'the'), 22086), ((',', 'and'), 21049), (('from', 'the'), 18931), (('.', 'He'), 17215), ((',', 'who'), 17083), (('to', 'be'), 16666), (('.', 'It'), 16413), (('Premier', 'League'), 16392), ((',', 'the'), 16159), (('the', 'cl

Can see examples of generic stuff that don't really want - but mostly have

* Titles and summaries that are not particularly interesting
* Texts with generic filler stuff, that can get rid of

I think maybe better off splitting into sentences to remove

Use `Counter` method to find common sentences that are shitty and we replace 

Note that I guess we just replace them here to make the visual analysis easier - because it really isn't that important for the sake of analysis to remove this info. If we wanted to remove it, we would have to develop some automatic system
- I guess some TFIDF style approach on sentences would be quite good
- Maybe should just remove all sentences that are above a certain count

Maybe should leave this for now until we want to do story summaries, as by that time we will probably have a very different looking set of texts

In [59]:
c = reduced_stories.story_text.notna()
story_texts = reduced_stories.loc[c, 'story_text'].tolist()
sentences = [sentence.strip() for story in story_texts for sentence in re.split('(?:\\n|[\.?!])', story) if sentence != '']
sentence_count = Counter(sentences)

In [64]:
sentence_count.most_common(5)

[('Share this with', 9408),
 ('', 7385),
 ('Attempt missed', 4967),
 ('4', 4612),
 ('Attempt saved', 3187)]

Maybe the thing to do is to move directly to use spacy to try and tag stuff and see what we have to play with - correct this, build a model, then test that model on the texts etc.

https://spacy.io/usage/linguistic-features#named-entities

Note that there may be some odd things that we have to remove before extracting entities, like punctuation

For now - just go through and pull the entities and start iterating to make the job easier.
The task should be to move through and correct, so that we can correct

In [6]:
import spacy
disable = ['parser', 'textcat']
nlp = spacy.load('en_core_web_lg', disable = disable)

def is_stop(token):
    try:
        return token.text.lower() in nlp.Defaults.stop_words
    except:
        return False

In [7]:
i = 0
title_entities = []
text_list = reduced_stories.article_title.tolist()
unwanted_ents = ['DATE', 'ORDINAL', 'CARDINAL', 'TIME']

# Remove all bad
regex_replace = '[^A-zÀ-ÿ0-9\- ]'

# Remove all non-alpha from start and end
regex_strip_start = '^[^A-zÀ-ÿ0-9]*'
regex_strip_end = '[^A-zÀ-ÿ0-9]*$'

fix_ent = lambda ent: re.sub(regex_strip_end, '', re.sub(regex_strip_start, '', re.sub(regex_replace, '', ent.replace("'s", ''))))

for doc in nlp.pipe(text_list, batch_size = 10000, n_threads = 4):
    if i % 1000 == 0:
        print('Done with {} of {} docs'.format(i, len(text_list)))
    
    # Not particularly interested in location as we want to count and normalise
    treated_text = [(fix_ent(ent.text), ent.label_) for ent in doc.ents if ent.label_ not in unwanted_ents]
    title_entities.append(treated_text)
    i += 1

Done with 0 of 18524 docs
Done with 1000 of 18524 docs
Done with 2000 of 18524 docs
Done with 3000 of 18524 docs
Done with 4000 of 18524 docs
Done with 5000 of 18524 docs
Done with 6000 of 18524 docs
Done with 7000 of 18524 docs
Done with 8000 of 18524 docs
Done with 9000 of 18524 docs
Done with 10000 of 18524 docs
Done with 11000 of 18524 docs
Done with 12000 of 18524 docs
Done with 13000 of 18524 docs
Done with 14000 of 18524 docs
Done with 15000 of 18524 docs
Done with 16000 of 18524 docs
Done with 17000 of 18524 docs
Done with 18000 of 18524 docs


In [11]:
entity_results = pd.DataFrame({
    'headline' : text_list,
    'entities' : title_entities
})

In [13]:
entity_results.to_excel(save_loc / 'headline_entities_v0.xlsx', index = False)

In [8]:
title_entities

[[('Son Hueng-min', 'PERSON'), ('Tottenham', 'PERSON'), ('Spurs', 'ORG')],
 [('Burnley', 'PERSON'), ('Sean Dyche', 'PERSON')],
 [('Wes Morgan', 'PERSON'),
  ('Leicester', 'GPE'),
  ('Brendan Rodgers', 'PERSON')],
 [('the Premier League', 'ORG'), ('London', 'GPE')],
 [('James Forrest', 'PERSON'), ('Rangers', 'ORG'), ('Celtic', 'NORP')],
 [('Watford', 'ORG'), ('Ole Gunnar Solskjaer', 'PERSON')],
 [('Paul Gascoigne', 'PERSON'),
  ('Spurs', 'ORG'),
  ('Tottenham', 'GPE'),
  ('Inter Milan', 'ORG')],
 [('Manchester City', 'GPE'), ('Fulham', 'GPE'), ('Premier League', 'ORG')],
 [('Pierre-Emile Hojbjerg', 'PERSON'),
  ('Southampton', 'GPE'),
  ('Brighton', 'GPE')],
 [('Phil Neville', 'PERSON'),
  ('Lucy Bronze', 'PERSON'),
  ('England', 'GPE'),
  ('World Cup', 'EVENT')],
 [('Everton', 'ORG'), ('Michael Keane', 'PERSON')],
 [('Michy Batshuayi', 'PERSON'),
  ('Crystal Palace', 'FAC'),
  ('Burnley', 'PERSON')],
 [('Beyonce', 'ORG'),
  ('Rowling', 'PERSON'),
  ('Mother Theresa', 'PERSON'),
  ('Eng

In [78]:
title_entities[:2]

[[('Son Hueng-min', 'PERSON'), ('Tottenham', 'PERSON'), ('Spurs', 'ORG')],
 [('Burnley', 'PERSON'), ('Sean Dyche', 'PERSON')]]

I think move to count the entities, and count things to try and normalise

* Things like Tottenham and Spurs could be related to be together, to group
* I think grouping things together might be a good way in general to normalise
* Note that once we have normalised, we should run an extract / replace - so that we avoid errors in the tagger
* Will have to shave off some things like 's
* Also have slight issue in defining whether someone is a person or player or manager - maybe not super important for now
    * This could be classified using context
    * Or maybe defined, again, using a network
    
To do:

* See what the common entities are (considered as singles)
* See what common "n gram" entities are (Tottenham Spurs)
* See what connections we have 

In [79]:
all_ents = []
_ = [all_ents.append(ent) for title_ents in title_entities for ent in title_ents]

In [80]:
all_ents[0:5]

[('Son Hueng-min', 'PERSON'),
 ('Tottenham', 'PERSON'),
 ('Spurs', 'ORG'),
 ('Burnley', 'PERSON'),
 ('Sean Dyche', 'PERSON')]

In [81]:
ent_frame = pd.DataFrame(all_ents, columns = ['entity', 'tag'])

# Remove if entity is empty
ent_frame = ent_frame[ent_frame.entity != ''].reset_index(drop = True)

# Get the count of the combination
ent_frame['combination_count'] = ent_frame.groupby(['entity', 'tag'])['tag'].transform('count')

# Get the count of the entity alone
ent_frame['entity_count'] = ent_frame.groupby('entity')['tag'].transform('count')

# And drop all duplicates
ent_frame = ent_frame.drop_duplicates().reset_index(drop = True)

# And get one with most popular
only_ents = ent_frame.sort_values('entity_count', ascending = False).drop_duplicates('entity').reset_index(drop = True)

Have to check some stuff:

* How many only have 1
* How many are above a certain count (i.e. quantiles)

In [82]:
prct_1 = (only_ents.entity_count == 1).mean()
print('{:.0%} have only a single count'.format(prct_1))
print(only_ents.loc[only_ents.entity_count != 1, 'entity_count'].quantile([0.25, 0.5, 0.75, 0.9]))

60% have only a single count
0.25     2.0
0.50     4.0
0.75     9.0
0.90    25.0
Name: entity_count, dtype: float64


Treatments:

* Replace Man with manchester, utd with united

The goal here is to try and get some stuff that we can use as a base set of "good tags", so that we can train our own tagger that only takes into account the stuff we are interested in

* Don't know if need to create a new "train" set, where we have some stuff separated by spaces, or if we should just put everything as it is
* Don't want to get into a thing where we just pick up key terms

We need a well tagged set to relate tags between them to get "what" they are

* Maybe need to do an exercise of scraping teams and construct a dictionary of replacements / nick names to get everything normalised
    * Not super scalable, but the tags we are getting back here won't really work that well either
* Need to find a source where we can scrape this stuff
* I guess we could start with something relatively limited, and at least get the process down, so we can expand later

https://en.wikipedia.org/wiki/List_of_association_football_competitions

https://en.wikipedia.org/wiki/Premier_League
https://en.wikipedia.org/wiki/Primeira_Liga
https://en.wikipedia.org/wiki/Eredivisie
https://en.wikipedia.org/wiki/EFL_Championship
https://en.wikipedia.org/wiki/EFL_League_One
https://en.wikipedia.org/wiki/EFL_League_Two
https://en.wikipedia.org/wiki/La_Liga
https://en.wikipedia.org/wiki/Segunda_Divisi%C3%B3n
https://en.wikipedia.org/wiki/Bundesliga
https://en.wikipedia.org/wiki/Serie_A
https://en.wikipedia.org/wiki/Ligue_1

In [83]:
only_ents[only_ents.entity_count >= 25]

,entity,tag,combination_count,entity_count
0,Liverpool,GPE,1248,1249
1,Chelsea,PRODUCT,1,1092
2,Manchester United,ORG,806,1056
3,Manchester City,GPE,766,766
4,Premier League,ORG,652,652
5,Tottenham,GPE,449,629
6,Celtic,GPE,2,406
7,Barcelona,GPE,372,388
8,Newcastle,GPE,382,382
9,Everton,ORG,119,349
